In [12]:
import tqdm
from torch import nn
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero, Linear
import torch_geometric.transforms as T
from torch_geometric.data import HeteroData
from torch import Tensor
from torch_geometric.datasets import DBLP
import numpy as np
import random


# We initialize conference node features with a single one-vector as feature:
dataset = DBLP('./data/dblp', transform=T.Constant(node_types='conference'))
data = dataset[0]
data["author"].node_id = torch.arange(data["author"].x.shape[0])
data["paper"].node_id = torch.arange(data["paper"].x.shape[0])
print(data)


def set_seed(seed=42):
    #torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    #torch.use_deterministic_algorithms(True)
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)

HeteroData(
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057],
    node_id=[4057],
  },
  paper={
    x=[14328, 4231],
    node_id=[14328],
  },
  term={ x=[7723, 50] },
  conference={
    num_nodes=20,
    x=[20, 1],
  },
  (author, to, paper)={ edge_index=[2, 19645] },
  (paper, to, author)={ edge_index=[2, 19645] },
  (paper, to, term)={ edge_index=[2, 85810] },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (term, to, paper)={ edge_index=[2, 85810] },
  (conference, to, paper)={ edge_index=[2, 14328] }
)


In [13]:
# For this, we first split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, we use 70% of edges for message passing,
# and 30% of edges for supervision.
# We further want to generate fixed negative edges for evaluation with a ratio of 2:1.
# Negative edges during training will be generated on-the-fly.
# We can leverage the `RandomLinkSplit()` transform for this from PyG:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("author", "to", "paper"),
    rev_edge_types=("paper", "to", "author"), 
)

train_data, val_data, test_data = transform(data)


In [14]:
train_data["author", "to", "paper"].edge_label

tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [15]:
# In the first hop, we sample at most 20 neighbors.
# In the second hop, we sample at most 10 neighbors.
# In addition, during training, we want to sample negative edges on-the-fly with
# a ratio of 2:1.
# We can make use of the `loader.LinkNeighborLoader` from PyG:
from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
edge_label_index = train_data["author", "to", "paper"].edge_label_index
edge_label = train_data["author", "to", "paper"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("author", "to", "paper"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

next(iter(train_loader))

HeteroData(
  author={
    x=[784, 334],
    y=[784],
    train_mask=[784],
    val_mask=[784],
    test_mask=[784],
    node_id=[784],
    n_id=[784],
    num_sampled_nodes=[3],
  },
  paper={
    x=[6759, 4231],
    node_id=[6759],
    n_id=[6759],
    num_sampled_nodes=[3],
  },
  term={
    x=[2409, 50],
    n_id=[2409],
    num_sampled_nodes=[3],
  },
  conference={
    num_nodes=20,
    x=[20, 1],
    n_id=[20],
    num_sampled_nodes=[3],
  },
  (author, to, paper)={
    edge_index=[2, 2015],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[2015],
    num_sampled_edges=[2],
    input_id=[128],
  },
  (paper, to, author)={
    edge_index=[2, 2332],
    e_id=[2332],
    num_sampled_edges=[2],
  },
  (paper, to, term)={
    edge_index=[2, 8286],
    e_id=[8286],
    num_sampled_edges=[2],
  },
  (paper, to, conference)={
    edge_index=[2, 190],
    e_id=[190],
    num_sampled_edges=[2],
  },
  (term, to, paper)={
    edge_index=[2, 10025],
    e_id=[10025],
    num_sa

In [18]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_movie: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_movie).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for users and movies:
        self.paper_lin = torch.nn.Linear(4231, hidden_channels)
        self.lin_dict = torch.nn.ModuleDict()
        for node_type in data.node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)
        self.author_emb = torch.nn.Embedding(data["author"].num_nodes, hidden_channels)
        self.paper_emb = torch.nn.Embedding(data["paper"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        #x_dict = {
        #  "author": self.author_emb(data["author"].node_id),
        #  "paper": self.paper_lin(data["paper"].x) + self.paper_emb(data["paper"].node_id),
        #}
        x_dict = {
            node_type: self.lin_dict[node_type](x).relu_()
            for node_type, x in data.x_dict.items()
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["author"],
            x_dict["paper"],
            data["author", "to", "paper"].edge_label_index,
        )

        return pred

        
model = Model(hidden_channels=64)

print(model)

Model(
  (paper_lin): Linear(in_features=4231, out_features=64, bias=True)
  (lin_dict): ModuleDict(
    (author): Linear(-1, 64, bias=True)
    (paper): Linear(-1, 64, bias=True)
    (term): Linear(-1, 64, bias=True)
    (conference): Linear(-1, 64, bias=True)
  )
  (author_emb): Embedding(4057, 64)
  (paper_emb): Embedding(14328, 64)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (author__to__paper): SAGEConv(64, 64, aggr=mean)
      (paper__to__author): SAGEConv(64, 64, aggr=mean)
      (paper__to__term): SAGEConv(64, 64, aggr=mean)
      (paper__to__conference): SAGEConv(64, 64, aggr=mean)
      (term__to__paper): SAGEConv(64, 64, aggr=mean)
      (conference__to__paper): SAGEConv(64, 64, aggr=mean)
    )
    (conv2): ModuleDict(
      (author__to__paper): SAGEConv(64, 64, aggr=mean)
      (paper__to__author): SAGEConv(64, 64, aggr=mean)
      (paper__to__term): SAGEConv(64, 64, aggr=mean)
      (paper__to__conference): SAGEConv(64, 64, aggr=mean)
      (term__to__paper): SAG

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 6):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()

        sampled_data.to(device)
        pred = model(sampled_data)

        ground_truth = sampled_data["author", "to", "paper"].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'


  0%|          | 0/37 [00:00<?, ?it/s]

torch.Size([823, 64])
torch.Size([6614, 64])
{('author', 'to', 'paper'): EdgeIndex([[ 365,  366,  367,  ...,  359,  822,  363],
           [   1,    3,    5,  ..., 1858, 1859, 1859]], device='cuda:0',
          sparse_size=(823, 6614), nnz=2266, sort_order=col), ('paper', 'to', 'author'): EdgeIndex([[ 375,  376,  156,  ..., 2536, 2537,  374],
           [   0,    0,    1,  ...,  542,  542,  542]], device='cuda:0',
          sparse_size=(6614, 823), nnz=2503, sort_order=col), ('paper', 'to', 'term'): EdgeIndex([[2538,  579, 2539,  ..., 4405,  374,  374],
           [   0,    0,    0,  ..., 1036, 1036, 1037]], device='cuda:0',
          sparse_size=(6614, 2594), nnz=8088, sort_order=col), ('paper', 'to', 'conference'): EdgeIndex([[6424,  432, 6487, 2623, 6488, 6489, 6490, 6491, 6492, 6493, 6494,
            6495, 6496, 6477, 6497, 6498,   46, 6499, 6500, 4581, 6501, 6502,
            6503, 6504, 6505,   56, 6506, 6507,  382, 6508, 2320, 1689, 6509,
            6510, 6511, 6512, 6513, 651

  3%|▎         | 1/37 [00:00<00:07,  4.82it/s]

torch.Size([834, 64])
torch.Size([6519, 64])
{('author', 'to', 'paper'): EdgeIndex([[  30,  363,  364,  ...,  358,  359,  476],
           [   1,    2,    3,  ..., 1684, 1685, 1685]], device='cuda:0',
          sparse_size=(834, 6519), nnz=2107, sort_order=col), ('paper', 'to', 'author'): EdgeIndex([[ 380,  381,  382,  ..., 2453, 2454, 2455],
           [   0,    0,    0,  ...,  560,  560,  560]], device='cuda:0',
          sparse_size=(6519, 834), nnz=2427, sort_order=col), ('paper', 'to', 'term'): EdgeIndex([[2456, 1894, 2457,  ..., 1289, 6404,  379],
           [   0,    0,    0,  ..., 1038, 1038, 1038]], device='cuda:0',
          sparse_size=(6519, 2463), nnz=8140, sort_order=col), ('paper', 'to', 'conference'): EdgeIndex([[5920, 5228, 6405, 6406, 6407, 6408, 5503, 3050, 4105, 6409, 6410,
            3292, 4637, 6230, 6411, 5484, 6412, 6413, 2193, 6087, 6414, 3149,
            6415, 6416, 6417, 6418, 3148, 6419,  386, 6420,  989,  985, 5895,
            6421, 6422,  961, 6423, 642

...

  8%|▊         | 3/37 [00:00<00:01, 24.91it/s]

torch.Size([809, 64])
torch.Size([6603, 64])
{('author', 'to', 'paper'): EdgeIndex([[ 364,  365,  365,  ...,  361,  362,  363],
           [   2,    3,    4,  ..., 1709, 1710, 1711]], device='cuda:0',
          sparse_size=(809, 6603), nnz=2062, sort_order=col), ('paper', 'to', 'author'): EdgeIndex([[ 379,  380,  381,  ..., 2535, 2536, 2537],
           [   0,    0,    0,  ...,  565,  565,  565]], device='cuda:0',
          sparse_size=(6603, 809), nnz=2501, sort_order=col), ('paper', 'to', 'term'): EdgeIndex([[2538, 2539, 2540,  ..., 2466, 4130,  378],
           [   0,    0,    0,  ..., 1047, 1047, 1047]], device='cuda:0',
          sparse_size=(6603, 2382), nnz=8268, sort_order=col), ('paper', 'to', 'conference'): EdgeIndex([[3783, 6500, 6501, 3552, 6502, 6503, 6504, 2097, 5926,  513, 6505,
            6506, 6207, 6131,   46, 6507, 6508, 6509, 2439, 4551, 1769, 2906,
            6510, 6511, 6512, 6513, 2905, 6514, 1733, 6515, 6516, 1549, 6517,
            6518,   62, 1184, 6519, 652

 22%|██▏       | 8/37 [00:00<00:01, 23.09it/s]

{('author', 'to', 'paper'): EdgeIndex([[ 365,  366,   55,  ...,  362,  806,  363],
           [   0,    2,    3,  ..., 1760, 1760, 1761]], device='cuda:0',
          sparse_size=(807, 6433), nnz=2117, sort_order=col), ('paper', 'to', 'author'): EdgeIndex([[ 381,  382,  383,  ...,  651, 2258,  657],
           [   0,    0,    0,  ...,  552,  552,  552]], device='cuda:0',
          sparse_size=(6433, 807), nnz=2411, sort_order=col), ('paper', 'to', 'term'): EdgeIndex([[2466,    0, 2467,  ..., 2368, 2504,  380],
           [   0,    0,    0,  ..., 1003, 1003, 1003]], device='cuda:0',
          sparse_size=(6433, 2430), nnz=7881, sort_order=col), ('paper', 'to', 'conference'): EdgeIndex([[6310, 6311, 6312, 6313, 6314,  842,  841, 3287, 4413, 6315, 6316,
            6317, 6318, 6319,   52, 6320, 6321, 6322, 6323, 6324, 6325, 6326,
            6327, 6328, 1221, 6329, 6330, 6331, 6332,  382, 6333, 6334, 6335,
            6336, 6337, 2263, 6338, 6339, 6340, 6341, 6342, 2140, 4486, 6343,
      

KeyboardInterrupt: 